In [ ]:
%matplotlib inline
from pathlib import Path
from itertools import combinations
import sys, time
sys.path.append("..")
from importlib import reload

import numpy as np
import scipy as sp
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.optimize import curve_fit
import pandas as pd
from matplotlib import pyplot as plt
import igraph

import umap

from sklearn import metrics
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pythd

In [ ]:
def time_thd(nrows, ncols, ncover, filt, metric='euclidean', precompute=False):
    total_time = time.perf_counter()
    setup_time = total_time
    
    dataset = np.random.normal(size=(nrows, ncols))
    
    clustering = pythd.clustering.HierarchicalClustering(method='complete', 
                                                         metric=('precomputed' if precompute else metric))
    
    filt.reset()
    f_x = filt(dataset)
    
    cov = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, ncover, 0.6)

    thd = pythd.thd.THD(dataset, filt, cov, clustering=clustering, 
                    group_threshold=2, contract_amount=0.1, 
                    precompute=precompute, metric=metric)
    
    thd_time = time.perf_counter()
    setup_time = thd_time - setup_time
    
    thd.run()
    
    last_time = time.perf_counter()
    total_time = last_time - total_time
    thd_time = last_time - thd_time
    
    return (setup_time, thd_time, total_time)

def time_thds(niter, nrows, ncols, ncover, filt, metric='euclidean', precompute=False):
    total_times = []
    setup_times = []
    thd_times = []
    
    for i in range(niter):
        setup_time, thd_time, total_time = time_thd(nrows, ncols, ncover, filt, metric, precompute)
        total_times.append(total_time)
        setup_times.append(setup_time)
        thd_times.append(thd_time)
    
    return total_times, setup_times, thd_times
    
def confidence_interval(data, confidence=0.99):
    """Compute a confidence interval for a series of data.
    
    Returns half the width of the interval"""
    data = np.array(data)
    n = data.shape[0]
    sem = sp.stats.sem(data) # standard error of the mean
    return sem * sp.stats.t.ppf((1 + confidence) / 2.0, n - 1)

def format_times(tms):
    """Format a list of times using a 99% confidence interval"""
    tms = np.array(tms)
    mean = tms.mean()
    h = confidence_interval(tms)
    return "{:.4f} +/- {:.4f}".format(mean, h)

def get_functs():
    def linear(x, a, b):
        return x*a + b
    def quadrat(x, a, b, c):
        return x*(a*x + b) + c
    def cubic(x, a, b, c, d):
        return x*(x*(a*x + b)+c)+d
    def logarithmic(x, a, b, c):
        return a*np.log(b*x)+c
    def nlogn(x, a, b, c):
        return a*x*np.log(b*x)+c
    def exponential(x, a, b, c):
        return a*np.exp(b*x)+c
    def logistic(x, L, k, x0):
        return L / (1 + np.exp(-k*(x - x0)))
    
    # dictionary of functions with parameter bounds
    d = {
        'linear': (linear, 
                   (np.array([-np.inf]*2), np.array([np.inf]*2))),
        'quadratic': (quadrat,
                      (np.array([-np.inf]*3), np.array([np.inf]*3))),
        'cubic': (cubic,
                  (np.array([-np.inf]*4), np.array([np.inf]*4))),
        'logarithmic': (logarithmic,
                        (np.array([-np.inf, 0., -np.inf]), np.array([np.inf]*3))),
        'nlogn': (nlogn,
                  (np.array([-np.inf, 0., -np.inf]), np.array([np.inf]*3))),
        'exponential': (exponential,
                        (np.array([-np.inf]*3), np.array([np.inf, 10., np.inf]))),
        'logistic': (logistic,
                     (np.array([0., 0., -np.inf]), np.array([np.inf, np.inf, np.inf])))
    }
    return d
    
def fit_curves(x, y):
    d = get_functs()
    res = []
    
    for k, tup in d.items():
        f, bounds = tup
        try:
            popt, pcov = curve_fit(f, x, y, bounds=bounds, maxfev=2000)
            ssq = np.sum((y - f(x, *popt))**2)
            res.append((k, popt, ssq))
        except (RuntimeError, ValueError) as e:
            print(k, e)
    
    res.sort(key=lambda zz: zz[2], reverse=False)
    return res

In [ ]:
# varying dataset size
DATASET_SIZES = np.logspace(1, 3.8, num=60, base=10.0, dtype=int)
orig_means = []
orig_intervals = []
opt_means = []
opt_intervals = []

for sz in DATASET_SIZES:
    print("Dataset size", sz)
    filt = pythd.filter.ScikitLearnFilter(PCA, n_components=2)
    
    orig_times = time_thds(50, nrows=sz, ncols=2, ncover=10, filt=filt, precompute=False)[0]
    orig_means.append(np.mean(orig_times))
    orig_intervals.append(confidence_interval(orig_times, confidence=0.95))
    
    opt_times = time_thds(50, nrows=sz, ncols=2, ncover=10, filt=filt, precompute=True)[0]
    opt_means.append(np.mean(orig_means))
    opt_intervals.append(confidence_interval(opt_times, confidence=0.95))

In [ ]:
# plot varying dataset size
orig_means = np.array(orig_means)
orig_intervals = np.array(orig_intervals)
plt.plot(DATASET_SIZES, orig_means, "-", color='blue', label='unoptimized')
plt.fill_between(DATASET_SIZES, orig_means - orig_intervals, orig_means + orig_intervals, alpha=0.3, color='blue')

opt_means = np.array(opt_means)
opt_intervals = np.array(opt_intervals)
plt.plot(DATASET_SIZES, opt_means, '-', color='red', label='optimized')
plt.fill_between(DATASET_SIZES, opt_means - opt_intervals, opt_means + opt_intervals, alpha=0.3, color='red')
plt.legend(loc='best')
plt.xlabel('dataset size (rows)')
plt.ylabel('average runtime (s)')
plt.show()

# Logarithmic for smaller sizes, linear for larger? What is the turning point?
min_ssq = np.inf
functs = get_functs()
for thresh in DATASET_SIZES[5:-5]:
    below = DATASET_SIZES < thresh
    log_f, log_bound = functs['logarithmic']
    log_parms, _ = curve_fit(log_f, DATASET_SIZES[below], opt_means[below], bounds=log_bound, maxfev=2000)
    log_ssq = np.sum((opt_means[below] - log_f(DATASET_SIZES[below], *log_parms)) ** 2)

    lin_f, lin_bound = functs['linear']
    lin_parms, _ = curve_fit(lin_f, DATASET_SIZES[~below], opt_means[~below], bounds=lin_bound, maxfev=2000)
    lin_ssq = np.sum((opt_means[~below]))
    # Smallest sum-of-squares error so far
    if (log_ssq + lin_ssq) < min_ssq:
        min_ssq = log_ssq + lin_ssq
        best_thresh = thresh
        best_log_parms = log_parms
        best_lin_parms = lin_parms

print(best_thresh, min_ssq)
below = DATASET_SIZES < best_thresh

plt.plot(DATASET_SIZES, opt_means, '--', color='red', label='actual')
plt.plot(DATASET_SIZES[below], log_f(DATASET_SIZES[below], *best_log_parms), label='logarithmic')
plt.plot(DATASET_SIZES[~below], lin_f(DATASET_SIZES[~below], *best_lin_parms), label='linear')
plt.legend(loc='best')
plt.xlabel('dataset size (rows)')
plt.ylabel('average runtime (s)')
#plt.xscale('log')
#plt.yscale('log')
plt.show()

In [ ]:
# varying dataset dimension
DATASET_DIMS = np.array(range(10, 61))
orig_means = []
orig_intervals = []
opt_means = []
opt_intervals = []

for dim in DATASET_DIMS:
    print("Dataset dimension", dim)
    filt = pythd.filter.ScikitLearnFilter(PCA, n_components=2)
    
    orig_times = time_thds(50, nrows=1000, ncols=dim, ncover=10, filt=filt, precompute=False)[0]
    orig_means.append(np.mean(orig_times))
    orig_intervals.append(confidence_interval(orig_times, confidence=0.95))
    
    opt_times = time_thds(50, nrows=1000, ncols=dim, ncover=10, filt=filt, precompute=True)[0]
    opt_means.append(np.mean(orig_means))
    opt_intervals.append(confidence_interval(opt_times, confidence=0.95))

In [ ]:
# plot varying dataset dimension
orig_means = np.array(orig_means)
orig_intervals = np.array(orig_intervals)
plt.plot(DATASET_DIMS, orig_means, "-", color='blue', label='unoptimized')
plt.fill_between(DATASET_DIMS, orig_means - orig_intervals, orig_means + orig_intervals, alpha=0.3, color='blue')

opt_means = np.array(opt_means)
opt_intervals = np.array(opt_intervals)
plt.plot(DATASET_DIMS, opt_means, '-', color='red', label='optimized')
plt.fill_between(DATASET_DIMS, opt_means - opt_intervals, opt_means + opt_intervals, alpha=0.3, color='red')
plt.legend(loc='best')
plt.xlabel('dataset dimension (columns)')
plt.ylabel('average runtime (s)')

In [ ]:
# varying number of open sets in the cover
COVER_COUNTS = np.array(range(2, 51))
orig_means = []
orig_intervals = []
opt_means = []
opt_intervals = []

for ncov in COVER_COUNTS:
    print(ncov, "covers")
    filt = pythd.filter.ScikitLearnFilter(PCA, n_components=2)
    
    orig_times = time_thds(50, nrows=200, ncols=2, ncover=int(ncov), filt=filt, precompute=False)[0]
    orig_means.append(np.mean(orig_times))
    orig_intervals.append(confidence_interval(orig_times, confidence=0.95))
    
    opt_times = time_thds(50, nrows=200, ncols=2, ncover=int(ncov), filt=filt, precompute=True)[0]
    opt_means.append(np.mean(orig_means))
    opt_intervals.append(confidence_interval(opt_times, confidence=0.95))

In [ ]:
# plot varying open set numbers
orig_means = np.array(orig_means)
orig_intervals = np.array(orig_intervals)
plt.plot(COVER_COUNTS, orig_means, "-", color='blue', label='unoptimized')
plt.fill_between(COVER_COUNTS, orig_means - orig_intervals, orig_means + orig_intervals, alpha=0.3, color='blue')

opt_means = np.array(opt_means)
opt_intervals = np.array(opt_intervals)
plt.plot(COVER_COUNTS, opt_means, '-', color='red', label='optimized')
plt.fill_between(COVER_COUNTS, opt_means - opt_intervals, opt_means + opt_intervals, alpha=0.3, color='red')
plt.legend(loc='best')
plt.xlabel('number of open sets in cover')
plt.ylabel('average runtime (s)')